In [41]:
import pandas as pd
import numpy as np

from bokeh.models import ColumnDataSource, HoverTool, CategoricalColorMapper
from bokeh.plotting import figure
from bokeh.io import show, output_notebook, output_file

### Basics of Bokeh

In [2]:
# bokeh basics
# Create a blank figure with labels
p = figure(plot_width = 400, plot_height = 400, 
           title = 'Example Glyphs',
           x_axis_label = 'X', y_axis_label = 'Y')

# Example data
squares_x = [1, 3, 4, 5, 8]
squares_y = [8, 7, 3, 1, 10]
circles_x = [9, 12, 4, 3, 15]
circles_y = [8, 4, 11, 6, 10]

# Add squares glyph
p.square(squares_x, squares_y, size = 12, color = 'navy', alpha = 0.6)
# Add circle glyph
p.circle(circles_x, circles_y, size = 12, color = 'red')

# Set to output the plot in the notebook
output_notebook()
# Show the plot
show(p)

Loading BokehJS ...

In [3]:
# Read the data from a csv into a dataframe
flights = pd.read_csv('flights.csv', index_col=0)
# Summary stats for the column of interest
flights['arr_delay'].describe()

count    327346.000000
mean          6.895377
std          44.633292
min         -86.000000
25%         -17.000000
50%          -5.000000
75%          14.000000
max        1272.000000
Name: arr_delay, dtype: float64

In [4]:
flights.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
1,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00
2,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00
3,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00
4,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00
5,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00


In [5]:
flights.nunique()

year                 1
month               12
day                 31
dep_time          1318
sched_dep_time    1021
dep_delay          527
arr_time          1411
sched_arr_time    1163
arr_delay          577
carrier             16
flight            3844
tailnum           4043
origin               3
dest               105
air_time           509
distance           214
hour                20
minute              60
time_hour         6936
dtype: int64

In [6]:
"""Bins will be five minutes in width, so the number of bins is (length of interval / 5). 
Limit delays to [-60, +120] minutes using the range."""
arr_hist, edges = np.histogram(flights['arr_delay'], bins = int(180/5), range = [-60, 120])

# Put the information in a dataframe
delays = pd.DataFrame({'flights': arr_hist, 'left': edges[:-1], 'right': edges[1:]})

D:\Anaconda3\lib\site-packages\numpy\lib\histograms.py:839: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
D:\Anaconda3\lib\site-packages\numpy\lib\histograms.py:840: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


In [7]:
delays.head()

,flights,left,right
0,276,-60.0,-55.0
1,636,-55.0,-50.0
2,1394,-50.0,-45.0
3,2820,-45.0,-40.0
4,5339,-40.0,-35.0


In [8]:
# Create the blank plot
p = figure(plot_height = 600, plot_width = 600, title = 'Histogram of Arrival Delays', 
           x_axis_label = 'Delay (min)]', y_axis_label = 'Number of Flights')

# Add a quad glyph
p.quad(bottom=0, top=delays['flights'], left=delays['left'], right=delays['right'], 
       fill_color='red', line_color='black')

# Show the plot
show(p)

### Adding Interactivity

In [24]:
arr_df = pd.DataFrame({'count': arr_hist, 'left': edges[:-1], 'right': edges[1:]})
arr_df['f_count'] = ['%d flights' % count for count in arr_df['count']]
arr_df['f_interval'] = ['%d to %d minutes' % (left, right) for left, right in zip(arr_df['left'], 
                                                                                  arr_df['right'])]

arr_df.head()

,count,left,right,f_count,f_interval
0,276,-60.0,-55.0,276 flights,-60 to -55 minutes
1,636,-55.0,-50.0,636 flights,-55 to -50 minutes
2,1394,-50.0,-45.0,1394 flights,-50 to -45 minutes
3,2820,-45.0,-40.0,2820 flights,-45 to -40 minutes
4,5339,-40.0,-35.0,5339 flights,-40 to -35 minutes


In [25]:
# Import the ColumnDataSource class
from bokeh.models import ColumnDataSource
# Convert dataframe to column data source
src = ColumnDataSource(arr_df)
src.data.keys()

dict_keys(['index', 'count', 'left', 'right', 'f_count', 'f_interval'])

In [38]:
def style(p):
    p.title.align = 'center'
    p.title.text_font_size = '20pt'
    p.xaxis.axis_label_text_font_size = '12pt'
    #p.xaxis.major_label_text_font_size = '12pt'
    p.yaxis.axis_label_text_font_size = '12pt'
    #p.yaxis.major_label_text_font_size = '12pt'
    
    return p

In [39]:
# Set up the figure same as before
p = figure(plot_width = 600, plot_height = 600, title = 'Histogram of Arrival Delays', 
           x_axis_label = 'Delay(min)', y_axis_label = 'Flights')

# Add a quad glyph with source this time
p.quad(bottom=0, top='count', left='left', right='right', source=src, 
       fill_color='red', line_color='black', fill_alpha = 0.75, 
       hover_fill_alpha = 1.0, hover_fill_color = 'navy')

# Add style to the plot
styled_p = style(p)

# Add a hover tool referring to the formatted columns
hover = HoverTool(tooltips = [('Delay', '@f_interval'),
                              ('Count', '@f_count')])

# Add the hover tool to the graph
styled_p.add_tools(hover)

# Show the plot
show(styled_p)

In [42]:
output_file('hist.html')
show(p)